In [ ]:
#default_exp core.pauli

In [ ]:
#hide

from nbdev.showdoc import *
from nbdev.imports import *

In [ ]:
#export

from functools import cache
from typing import Tuple

import numpy as np

# Pauli

> Definition of the Pauli matrices, and useful methods for them.

In [ ]:
#export

"""Definition of the Pauli matrices."""
pauli_matrices = (
    np.identity(2),
    np.array([[0, 1], [1, 0]]),
    np.array([[0, -1j], [1j, 0]]),
    np.array([[1, 0], [0, -1]]),
)

pauli_names = ("I", "X", "Y", "Z")

In [ ]:
# Each Pauli squares to the identity
for i in range(4):
    test_eq(pauli_matrices[i] @ pauli_matrices[i], np.identity(2))

In [ ]:
#export

class PauliOperator:
    """
    Multi-qubit Pauli operator.
    
    The operator acts on a basis whose dimensions are indexed by self.indices. When written as
    a tensor product, the lowest index is on the right, corresponding to the least significant
    digit in a bitstring.
    """
    def __init__(self, indices: Tuple[int, ...]):
        """
        Initialize the operation from the indices of each Pauli matrix.
        
        index 0 = I
        index 1 = X
        index 2 = Y
        index 3 = Z
        """
        if any([idx not in {0, 1, 2, 3} for idx in indices]):
            raise ValueError(f"All indices must be in {{0, 1, 2, 3}}.")
        self.dim = 2 ** len(indices)
        self.indices = indices
    
    @property
    @cache
    def matrix(self):
        """
        Calculate and cache the matrix representation of this Pauli. Subsequent calls to 
        """
        if len(self.indices) > 16:
            raise ValueError(
                "Good grief, this is a didactic exercise. We won't allocate more than "
                f"16 qubits ({len(self.indices)} requested).")
        return reduce(
            lambda x, y: np.kron(x, y),
            reversed(
                list(
                    map(
                        lambda idx: pauli_matrices[idx],
                        self.indices
                    )
                )
            )
        )
    
    @staticmethod
    def name(indices: Tuple[int, ...]) -> str:
        """
        Compute the name of a PauliOperator with given indices. Note the written operator
        has the lowest index on the right.
        
        Example: indices (1, 3) -> "ZX"
        """
        return "".join(map(lambda x: pauli_names[x], reversed(indices)))

    def __repr__(self):
        return "PauliOperator " + "".join(map(lambda x: pauli_names[x], reversed(self.indices)))
    
    def __str__(self):
        return self.name(self.indices)


In [ ]:
p = PauliOperator((3, 1))

# PauliOperator.dim
test_eq(p.dim, 4)

# PauliOperator.matrix
test_eq(p.matrix, np.array([[0, 0, 1, 0], [0, 0, 0, -1], [1, 0, 0, 0], [0, -1, 0, 0]]))

# PauliOperator.__repr__
test_eq(p.__repr__(), "PauliOperator XZ")

# PauliOperator.__str__
test_eq(p.__str__(), "XZ")

# Indices have to be in {0, 1, 2, 3}
test_fail(lambda: PauliOperator(indices=(0, 1, 2, 3, 4)))

# Only up to 16 qubits allowed
p = PauliOperator(indices=(0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0))
test_fail(lambda: p.matrix)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted core.pauli.ipynb.
Converted index.ipynb.
Converted tomography.ipynb.
